# TP2 - Spatial Filtering
ATRIM - Option Datasim

Ecole Centrale Nantes

Diana Mateus

Participants: (FILL IN YOUR NAMES AND LASTNAMES)

# GOAL

In this lab we will practice the general principles of linear spatial filtering for 2D images:
- based on the convolution operation
- and applied with different filters to solve different tasks (denoising, highboost sharpening, border extraction)

Then, we will break some of the linear filtering assumptions:
- to find patterns in an image (Waldo/Charlie) with correlation,
- to filter preserving borders,
- to find corners in an image


# 0. Preparation

* Define the path to the images only once. Then, use the given path for the images

* Handling several large images can create large memory demands. In order to avoid large size files that slow processing, you can:
  - reuse the image variable names
  - clear large variables with the command
   ```reset_selective name_variable```
  - If required resize the images while debugging (e.g. to 100x100)
  - Before submiting your jupyter notebook empty the outputs first: go to the Kernel menu, restart and clear output.

  - If you get some warnings "IOPub data rate exceeded" launch your notebook with

  ```jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000```



### 0.1 Importing the required modules


In [ ]:
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np
import os
import math
from skimage.restoration import denoise_bilateral
from skimage.transform import resize, rescale
from scipy import ndimage
from scipy import signal


### 0.2 Define the main image folder
Make sure the subsequent parts of this notebook refer to this definition IMDIR. **When evaluating your notebook I should only need to change the path here** to run the entire notebook and find all the images

In [ ]:
IMDIR = "./images"


In [ ]:
#If using Colab
#from google.colab import drive
#drive.mount('/content/drive')
#IMDIR = "/content/drive/MyDrive/Colab Notebooks/2022-2023 Image Processing/02-spatial-filtering /images/"

### 0.3 Read and display the images
Check that you can read and display all the provided images. **Do not include this cell**, neither the code nor its ouput in the final report.

In [ ]:
width=20
height=10
plt.rcParams['figure.figsize'] = [width, height]
fig=plt.figure()

im_counter = 1
for root, dirnames, filenames in os.walk(IMDIR):
    #print(dirnames)
    for filename in filenames:
        f = os.path.join(root, filename)

        #filter only image files with the following format
        if f.endswith(('.png', '.jpg', '.jpeg','.JPG', '.tif', '.gif')):

            # print the paths to current filename if nothing is being found
            #print(filename)

            # read the image
            im = io.imread(f,as_gray=True)
            im = resize(im,(100,100),mode='constant')

            # display it
            if im_counter >= 25:
                break;

            plt.subplot(5,5,im_counter)
            plt.imshow(im, cmap='gray')
            plt.title(filename)
            plt.axis('off')
            im_counter +=1



plt.show()




# 1 Linear spatial filtering with convolution




## 1.1. Mean Kernel Example
The following ``meanKernel'' function creates a smoothing kernel, which can be used with scipy's ``` ndimage.convolve(im,kernel) ```
convolution function to blur an image.

Run the following cells to display different versions of the kernel and the application of a mean_kernel on an image.


In [ ]:
def meanKernel(hs):
    #hs defines the size of the kernel
    #hs is the integer half size of the kernel
    #creates an square filter with each side of length 2*hs+1
    kernel = np.zeros((hs*2+1,hs*2+1))
    kernel += 1/(hs*2+1)**2
    return kernel

#Display properties
width=12
height=3
plt.rcParams['figure.figsize'] = [width, height]

#creating and showing three mean kernels of different sizes
k = 1
for hs in [1,3,11]:
    plt.subplot(1,3,k)
    kernel = meanKernel(hs)
    plt.imshow(kernel, vmin=0, vmax=0.2)
    plt.title('Mean')
    plt.colorbar()
    k+=1
plt.show()


In [ ]:
f = os.path.join(IMDIR, "grass.jpg")

#Display properties
width=10
height=5

#Filter parameters
hs = 3
sigma = 2

# Read and preprocess image
im = io.imread(f, as_gray=True)
im = im.astype(float)
im = resize(im,(100,100))

# Define filter and convolve
kernel = meanKernel(hs)
im_filtered_scipy = ndimage.convolve(im,kernel)

# Display the original image
fig=plt.figure(figsize=(width, height))
plt.subplot(1,2,1)
plt.imshow(im, cmap = 'gray')
plt.title('Original')

# Display the filtered image
plt.subplot(1,2,2)
plt.imshow(im_filtered_scipy, cmap = 'gray')
plt.title('Mean scipy conv')


## 1.2. Gaussian Kernel
Following the above example, create, display and apply several (at least 3) Gaussian kernels with varying window size and standard deviation.

**Question: ** what is the sum of the kernel elements in each case?, why is this necessary?

```Hints```:
- use the 'None' or 'Nearest' interpolation options of imshow to display the kernel images

In [ ]:
#both versions work, pick the one you are more confortable with

def gaussianKernel(hs,sigma, normalize=True): #half window size and Gaussian sigma
    # creating an empty kernel
    kernel = np.zeros((hs*2+1,hs*2+1))
    ax = np.arange(-hs, hs+1)

    # filling the kernel elements
    xx, yy = np.meshgrid(ax, ax)

    ### FILL BEGIN
    kernel = np.exp( ) #FILL IN
    ### FILL END

    # normalize and return
    if normalize:
        kernel = kernel / np.sum(kernel)

    return kernel

def gaussianLambdaKernel(size,sigma, normalize=True): #full window size and Gaussian sigma

    ### FILL BEGIN
    kernel = np.fromfunction(lambda x, y: math.e ** ( FILL IN ), (size, size))
    ### FILL END

    if normalize:
        kernel = kernel / np.sum(kernel)

    return kernel


In [ ]:
f = os.path.join(IMDIR, "grass.jpg")

#Display size
width=10
height=10

# Read and preprocess image
im = io.imread(f, as_gray=True)
im = im.astype(float)
im = resize(im,(100,100))

# Display the original image
fig=plt.figure(figsize=(3, 3))
plt.imshow(im, cmap = 'gray')
plt.title('Original')

# Define filter parameters
filter_sizes=[1,3,5]
sigma_values=[0.1,1,2]

# Display filters as images
fig=plt.figure(figsize=(width, height))

#FILL IN

plt.show()

# Convolve the image with the filter and display the filtered image
fig=plt.figure(figsize=(width, height))

#FILL IN

plt.show()







## 1.3 Filtering with your own Convolution
**a)** Repeat the smoothing above with your own implementation of the ```convolution``` function. The function should receive as input an image and a filter kernel (matrix of weights) and return the filtered image. Compare your results with those from the scikit in-built function.

**b)** Apply a Gaussian filter of kernel size 7x7 (hs=3) and sigma 2 display side by side the results of your convolution vs. those of the in-built function to check your implementation is correct. Clearly state on the title of the image which version of the convolution function is being used.

**c)** **Write down your findings**, notably the reasons for any possible difference with the in-built implementation.

**d)** Why and how can the convolution can be written as a matrix multiplication? why is it interesting?
```Hint```:  see http://cs231n.github.io/convolutional-networks/

In [ ]:
def myConvolution(imsource,kernel):

    # Find image and kernel sizes
    im_shape = imsource.shape
    imh,imw = im_shape[0], im_shape[1]
    kh,kw = kernel.shape
    delta_h=int((kh-1)/2)
    delta_w=int((kw-1)/2)

    # Image padding
    imPadded = np.zeros((imh+2*delta_h,imw+2*delta_w))
    imPadded[delta_h:imh+delta_h,delta_w:imw+delta_w]=imsource

    # Create an empty image to store the result
    imDest = np.zeros((imh,imw))

    #BEGIN FILL IN
    for i in range(imh):
        for j in range(imw):


    #END FILL IN
    return imDest


In [ ]:
f = os.path.join(IMDIR, "grass.jpg")

#Display properties
width=10
height=10

# Read and preprocess image
im = io.imread(f, as_gray=True)
im = im.astype(float)
im = resize(im,(100,100))

# Display the original image
fig=plt.figure(figsize=(width, height))
plt.subplot(1,4,1)
plt.imshow(im, cmap = 'gray')
plt.title('Original')

# Define filter parameters
hs=3
sigma=2
kernel = gaussianKernel(hs,sigma)

# Convolve and display the filtered image


## 1.4 Derivative filters

Define the required kernel function and convolve them with the *AscentB* or *Moon* images in the ``enhance`` folder to obtain
*  the gradient image in the horizontal direction
*  the gradient image in the vertical direction
*  the Laplacian of the image
*  the sharpened image after addition of the Laplacian "details" (and normalization)


In [ ]:
def sobel_x():
    kernel = np.zeros((3,3))


    return kernel

def sobel_y():
    kernel = np.zeros((3,3))

    return kernel

def laplacian():
    kernel = np.ones((3,3))


    return kernel

def normalize(im):
    im = (im-im.min())/(im.max()-im.min())
    return im


In [ ]:
f = os.path.join(IMDIR, "moon-blurred.tif")
#f = os.path.join(IMDIR, "ascentB.png")

#Display properties
width=15
height=5

# Read and preprocess image
im = io.imread(f, as_gray=True)
im = im.astype(float)
#im = resize(im,(100,100))

# Display the original image
fig=plt.figure(figsize=(width, height))
plt.subplot(1,5,1)
plt.imshow(im, cmap = 'gray')
plt.title('Original')

# Convolve and display the filtered and the enhanced image


# 2 Non linear filtering

Choose one among the two following tasks: either finding Charlie or bilateral filter.

Hint: USE A DIFFERENT NOTEBOOK FOR WALDO

## 2.1 Correlation: Finding Charlie

Use patch-wise Normalized Cross Correlation (NCC) to automatically find Waldo (Charlie) in an image. To this end, look for the template image (``charlie-template``) inside ``marche-crop`` or the ``marche`` images. As the process can be long start with the cropped version, you might also find it useful to **create a separate notebook for this task only** as it is memory consuming. Evaluate the NCC expression from the slides (described in the non-local means advance filtering) to compare the template with every location in the target image, store the results and retrieve the location with the highest NCC score. Draw this location on the target image.

**Describe the process assumptions and limitations**

Hint: Start from your convolution operation but apply locally normalized correlation at each pixel



## 2.2 Bilateral Filter  

**a)** Implement your own version of the ``bilateral`` filter and compare its results vs. scikit ``denoise_bilateral`` function.

**b)** Compare and comment the bilateral results versus the mean and gaussian filter for the ``zebra`` group of images of the ``bilateral`` folder


In [ ]:
from skimage.restoration import denoise_bilateral
#im_filtered_scipy = denoise_bilateral(im, win_size=hs*2+1, sigma_color=0.05, sigma_spatial=15)


# 3. Harris corners

3.1 Implement the Harris corner detector.

**a)** Compute and show the gradient images ($Ix$ and $Iy$)  as well as their outer products ($IxIx$, $IxIy$, $I_yI_y$) for the *pixel-pancho* images in the ``corners`` folder

**b)** From the previous result, compute the **weighted** and **aggregated** gradient products. *Hints:*
- To weigh, convolve the gradient products ($IxIx$, $IxIy$, and $I_yI_y$) each with a Gaussian filter.
- To aggregate, sum up at each pixel location the values of the weighed gradient products over a window ($(x,y)\in\mathcal{N}_{x_0,y_0}$).

**c)** From the previous result,
- build the structure tensor or Harris matrix $M$ for each pixel $(x_0,y_0)$ in the image.
- and compute the cornerness criteria based on its eigenvalues. Display the result as a *score map* image.
Hint: Compute the cornerness implicitly (using the fact that determinant of a 2x2 matrix is the product of the matrix eigenvalues and the trace its sum)

**d)** Define a threshold on the score map, and display the detected corners on top of the original images




3.2 **Questions**
**a)** Why is the Gaussian filtering important?

**b)** Explain how steps 3.1 b) and 3.2 c) are related to the autocorrelation

**c)** What do the eigenvalues of the structure tensor matrix represent?

**d)** What is the influence of the k parameter?

**e)** What other parameters determine the result?

**f)** Observing the results for the two images, why is it interesting to extract the corners?